In [1]:
%run ../models/model_vgg19_FC.ipynb
%run ../dataset/dataset.ipynb
%run utils.ipynb
%run ../loss/loss.ipynb

D:\ProgramData\Anaconda3\envs\mlenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
import torchvision.transforms as transforms
import torch.optim as optim
import torchvision.transforms.functional as FT
from tqdm import tqdm
from torch.utils.data import DataLoader
# from utils import (
#     non_max_suppression,
#     mean_average_precision,
#     intersection_over_union,
#     cellboxes_to_boxes,
#     get_bboxes,
#     plot_image,
#     save_checkpoint,
#     load_checkpoint,
# )
# from loss import YoloLoss
from tqdm import tqdm

In [3]:
seed = 123
torch.manual_seed(seed)

# Hyperparameters etc. 
LEARNING_RATE = 2e-5
DEVICE = "cuda" if torch.cuda.is_available else "cpu"
BATCH_SIZE = 16 # 64 in original paper but I don't have that much vram, grad accum?
WEIGHT_DECAY = 0
EPOCHS = 300
# NUM_WORKERS = 2
PIN_MEMORY = True
LOAD_MODEL = False
LOAD_MODEL_FILE = "overfit.pth.tar"
IMG_DIR = "../dataset/data/preprocessed/training_images/"
LABEL_DIR = "../dataset/labels/"

In [4]:
class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, img, bboxes):
        for t in self.transforms:
            img, bboxes = t(img), bboxes

        return img, bboxes


transform = Compose([transforms.Resize((448, 448)), transforms.ToTensor(),])


def train_fn(train_loader, model, optimizer, loss_fn):
    mean_loss = []

    for batch_idx, (x, y) in enumerate(train_loader):
        x, y = x.to(DEVICE), y.to(DEVICE)
        out = model(x)
        loss = loss_fn(out, y)
        mean_loss.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    mean_loss = sum(mean_loss)/len(mean_loss)
    print(f"Mean loss was {mean_loss}")
    return mean_loss
    
    
def main():
    model = Yolov1(split_size=7, num_boxes=2, num_classes=1).to(DEVICE)
    optimizer = optim.Adam(
        model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY
    )
    loss_fn = YoloLoss(S=7, B=2, C=1)

    if LOAD_MODEL:
        load_checkpoint(torch.load(LOAD_MODEL_FILE), model, optimizer)

    train_dataset = VOCDataset(
        "../dataset/train_map.csv",# link of the train.csv file
        transform=transform,
        img_dir=IMG_DIR,
        label_dir=LABEL_DIR,
    )

    test_dataset = VOCDataset(
        "../dataset/val_map.csv", 
        transform=transform, 
        img_dir=IMG_DIR, 
        label_dir=LABEL_DIR,
    )

    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=BATCH_SIZE,
        # num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        shuffle=True,
        drop_last=True,
    )

    test_loader = DataLoader(
        dataset=test_dataset,
        batch_size=BATCH_SIZE,
        # num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        shuffle=True,
        drop_last=True,
    )
    
    EARLY_STOPPING_LIMIT = 10
    epochs_since_last_improvement = 0
    best_loss = float('inf')
    
    for epoch in tqdm(range(EPOCHS)):
        # for x, y in train_loader:
        #    x = x.to(DEVICE)
        #    for idx in range(8):
        #        bboxes = cellboxes_to_boxes(model(x))
        #        bboxes = non_max_suppression(bboxes[idx], iou_threshold=0.5, threshold=0.4, box_format="midpoint")
        #        plot_image(x[idx].permute(1,2,0).to("cpu"), bboxes)

        #    import sys
        #    sys.exit()

        pred_boxes, target_boxes = get_bboxes(
            train_loader, model, iou_threshold=0.5, threshold=0.4
        )

        mean_avg_prec = mean_average_precision(
            pred_boxes, target_boxes, iou_threshold=0.5, box_format="midpoint"
        )
        print(f"Train mAP: {mean_avg_prec}")

        #if mean_avg_prec > 0.9:
        #    checkpoint = {
        #        "state_dict": model.state_dict(),
        #        "optimizer": optimizer.state_dict(),
        #    }
        #    save_checkpoint(checkpoint, filename=LOAD_MODEL_FILE)
        #    import time
        #    time.sleep(10)

        mean_loss = train_fn(train_loader, model, optimizer, loss_fn)
        
        if mean_loss < best_loss:
            best_loss = mean_loss
            epochs_since_last_improvement = 0
        else:
            epochs_since_last_improvement += 1
            if epochs_since_last_improvement >= EARLY_STOPPING_LIMIT:
                print(f"Early stopped at epoch {epoch}!")
                print(f"Best model at epoch {epoch-epochs_since_last_improvement}!")
                break
#                 model.load_state_dict(best_model)
#                 break


if __name__ == "__main__":
    main()

  0%|                                                                                          | 0/300 [00:00<?, ?it/s]

Train mAP: 0.0


  0%|▎                                                                               | 1/300 [00:21<1:45:43, 21.22s/it]

Mean loss was 158.46241675482855
Train mAP: 0.0


  1%|▌                                                                               | 2/300 [00:37<1:30:01, 18.13s/it]

...

Mean loss was 1.1015777521663241
Train mAP: 0.9785105586051941


 15%|███████████▊                                                                   | 45/300 [12:21<1:09:02, 16.24s/it]

Mean loss was 1.0661610745721393
Train mAP: 0.98687344789505


 15%|███████████▊                                                                   | 45/300 [12:37<1:11:34, 16.84s/it]

Mean loss was 0.9507018253207207
Early stopped at epoch 45!
Best model at epoch 35!
